---
# Desafio 2

En el archivo main.py se desarrollo el preprocesamiento de un libro de academico del area de telecomunicaciones (Internet Congestion Control). Esto tuvo una complicacion extra, ya que el procesamiento de PDF suele ser mas complicado que procesar otro tipo de extenciones como .txt. Se filtro gran parte del princpio y final para excluir partes del libro que no tienen contenido propiamente de la materia en estudio (texto mismo de la estructura del libro, referencias), tambien en lo posible todo tipo de texto relacionado con la estrucutura misma del libro que no aportan al contenido de la materia (figuras, tablas, etc.). Claramente esta estapa es critica y afecta a la calidad de los embeddings finales. Esto llevó varias iteraciones, tratando de limpiar y obtener el mejor corpus posible para obtener los tokens. 

Si el modelo funciona bien, deberiamos ver cosas como:

most_similar("tcp")

- debería devolver “connection”, “retransmission”, “segment”, “syn”, “ack”, “sender”, “receiver”.

most_similar("congestion")

- “avoidance”, “window”, “flow”, “rate”, “increase”, “traffic”.

most_similar("loss")

- “retransmission”, “timeout”, “duplicate”, “ack”, “delay”.

most_similar("window")

- “cwnd”, “increase”, “slowstart”, “threshold”.


In [202]:
import os
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec
from tensorflow.keras.preprocessing.text import text_to_word_sequence

import plotly.express as px
from sklearn.manifold import TSNE



In [203]:
sentence_tokens = []
for _, row in df.iterrows():
    tokens = text_to_word_sequence(row["texto"])
    if tokens:
        sentence_tokens.append(tokens)

print("Ejemplo de tokens:", sentence_tokens[:2])
print("Total de oraciones tokenizadas:", len(sentence_tokens))

Ejemplo de tokens: [['one', 'tcp', '’', 'critical', 'function', 'congestion', 'control', 'objective', 'book', 'provide', 'overview', 'topic', 'special', 'emphasis', 'analytical', 'modeling', 'congestion', 'control', 'protocol'], ['tcp', '’', 'congestion', 'control', 'algorithm', 'ha', 'described', 'largest', 'human', 'made', 'feedback', 'controlled', 'system', 'world']]
Total de oraciones tokenizadas: 326


In [204]:
class callback(CallbackAny2Vec):
    """Callback para imprimir pérdida en cada epoch."""
    def __init__(self):
        self.epoch = 0
        self.loss_previous_step = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print(f"Loss after epoch {self.epoch}: {loss}")
        else:
            print(f"Loss after epoch {self.epoch}: {loss}")
        self.epoch += 1
        self.loss_previous_step = loss


In [205]:
w2v_model = Word2Vec(
    min_count=3,       # probar distintos: 1,2,3 según corpus
    window=4,
    vector_size=200,   
    negative=15,
    workers=4,
    sg=1,              # Skip-Gram
    seed=42
)

In [206]:
print("\nConstruyendo vocabulario...")
w2v_model.build_vocab(sentence_tokens)

print("Docs en el corpus:", w2v_model.corpus_count)
print("Vocabulario:", len(w2v_model.wv.index_to_key))



Construyendo vocabulario...
Docs en el corpus: 326
Vocabulario: 398


In [218]:
print("\nEntrenando Word2Vec...\n")

w2v_model.train(
    sentence_tokens,
    total_examples=w2v_model.corpus_count,
    epochs=10,
    compute_loss=True,
    callbacks=[callback()]
)


Entrenando Word2Vec...

Loss after epoch 0: 36990.89453125
Loss after epoch 1: 73668.25
Loss after epoch 2: 109920.6484375
Loss after epoch 3: 145187.921875
Loss after epoch 4: 180563.140625
Loss after epoch 5: 215340.8125
Loss after epoch 6: 249127.671875
Loss after epoch 7: 283384.03125
Loss after epoch 8: 316553.375
Loss after epoch 9: 349981.125


(27402, 52090)

In [219]:
def try_word(word):
    if word in w2v_model.wv:
        print(f"\nPalabras más similares a '{word}':")
        print(w2v_model.wv.most_similar(positive=[word], topn=10))
    else:
        print(f"\n❌ La palabra '{word}' NO está en el vocabulario.")

try_word("tcp")
try_word("congestion")
try_word("loss")
try_word("window")



Palabras más similares a 'tcp':
[('analytical', 0.941254198551178), ('significant', 0.9339608550071716), ('legacy', 0.9295880198478699), ('describe', 0.9277751445770264), ('similar', 0.925797700881958), ('main', 0.9252133965492249), ('define', 0.9232828617095947), ('introduction', 0.9178593158721924), ('active', 0.9127635955810547), ('without', 0.9094575643539429)]

Palabras más similares a 'congestion':
[('procedure', 0.9080710411071777), ('idea', 0.9033729434013367), ('control', 0.9028500318527222), ('described', 0.9003563523292542), ('inspired', 0.896904468536377), ('tfrc', 0.8921334743499756), ('algorithm', 0.8850215077400208), ('without', 0.8725646138191223), ('linux', 0.8722782731056213), ('default', 0.8643463253974915)]

Palabras más similares a 'loss':
[('queue', 0.9296297430992126), ('distribution', 0.9035860300064087), ('random', 0.8958392143249512), ('size', 0.8866466879844666), ('estimated', 0.8854783773422241), ('threshold', 0.8711441159248352), ('given', 0.86946016550064

In [220]:
def reduce_dimensions(model, dim=2):
    vecs = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)
    tsne = TSNE(n_components=dim, random_state=0)
    reduced = tsne.fit_transform(vecs)
    return reduced, labels

In [221]:
print("\nGenerando visualización 2D...")
vecs2d, labels = reduce_dimensions(w2v_model)

MAX_WORDS = 200

fig = px.scatter(
    x=vecs2d[:MAX_WORDS, 0],
    y=vecs2d[:MAX_WORDS, 1],
    text=labels[:MAX_WORDS]
)
fig.update_traces(marker_size=7)
fig.show()


Generando visualización 2D...


In [211]:
print("Generando visualización 3D...")

vecs3d, labels = reduce_dimensions(w2v_model, dim=3)

fig = px.scatter_3d(
    x=vecs3d[:MAX_WORDS, 0],
    y=vecs3d[:MAX_WORDS, 1],
    z=vecs3d[:MAX_WORDS, 2],
    text=labels[:MAX_WORDS]
)
fig.update_traces(marker_size=3)
fig.show()

Generando visualización 3D...


In [212]:
print("\nGuardando vectors.tsv y labels.tsv...")

np.savetxt("tcp_vectors.tsv", w2v_model.wv.vectors, delimiter="\t")

with open("tcp_labels.tsv", "w") as fp:
    for token in w2v_model.wv.index_to_key:
        fp.write(token + "\n")

print("Listo. Archivos exportados.")




Guardando vectors.tsv y labels.tsv...
Listo. Archivos exportados.


La conclusiones nos son muy positivas, luego de largas iteraciones sobre el preprocesamiento, no logre encontrar un mejor pipeline. Con ello las palabras similares para los conceptos nombrados al inicio, no tienen correspondencia. Encuentro que hay valores extremadamente grandes de similud > 0.95 entre las palabras elegidas y palabras del vocabulario que no tienen conexion o relacion.

[('analytical', 0.941254198551178), ('significant', 0.9339608550071716), ('legacy', 0.9295880198478699), ('describe', 0.9277751445770264), ('similar', 0.925797700881958), ('main', 0.9252133965492249), ('define', 0.9232828617095947), ('introduction', 0.9178593158721924), ('active', 0.9127635955810547), ('without', 0.9094575643539429)]

Son match con palabras sin aporte y podrian considerarse wordstops que esten altamente repetidas a lo largo del curpur que no permita un buen entrenamiento con la tecnica Skip-Gram. 

Pero por otro lado, cuando se analiza visualmente, hay indicios de que el camino es el correcto. Observando agrupaciones como gateway, client, aks, como buen agrupamiento de vectores de embeddings. 